In [1]:
import sys
from scipy.io import loadmat
import matplotlib.pyplot as plt
sys.path.append('../')
sys.path.append('../util/')
from anomaly_detectors import *
from get_RGB import get_RGB

In [2]:
"""
Demo script that runs all anomaly detectors in hsi_toolkit_py using example sub MUUFL Gulfport data

Inputs:
	hsi_img_sub - n_row x n_col x n_band hyperspectral image
	wavelengths - n_band x 1 vector listing wavelength values for hsi_img in nm
	mask_sub - n_row x n_col binary image limiting detector operation to pixels where mask is true
		       if not present or empty, no mask restrictions are used
Outputs:
	det_out - dictionary of detector output images

5/5/2018 - Alina Zare
10/1/2018 - Python Implementation by Yutai Zhou
"""
an_hsi_image_sub_for_demo = loadmat('an_hsi_image_sub_for_demo.mat')
hsi_img_sub = an_hsi_image_sub_for_demo['hsi_img_sub']
wavelengths = an_hsi_image_sub_for_demo['wavelengths']
mask_sub = an_hsi_image_sub_for_demo['mask_sub']

det_out = {}
det_out['RGB'] = get_RGB(hsi_img_sub, wavelengths)
det_out['Valid Mask'] = mask_sub

# init detector args
guard_win = 3; bg_win = 3; n_dim_ss = 3

In [3]:
def gmrx_anomaly(hsi_img, n_comp, mask = None):
    """
    Gaussian Mixture RX Anomaly Detector
    fits GMM assuming entire image is background
    assigns pixels to highest posterior probability mixture component
    computes pixel Mahlanobis distance to component mean

    Inputs:
    hsi_image - n_row x n_col x n_band hyperspectral image
    mask - binary image limiting detector operation to pixels where mask is true
       if not present or empty, no mask restrictions are used
    n_comp - number of Gaussian components to use

    Outputs:
     gmrx_out - detector output image

    8/7/2012 - Taylor C. Glenn - tcg@cise.ufl.edu
    5/5/2018 - Edited by Alina Zare
    11/2018 - Python Implementation by Yutai Zhou
    """
    gmm_out, kwargsout = img_det(gmrx_helper, hsi_img,  , mask, n_comp = n_comp)
    return gmm_out

def gmrx_helper(hsi_data, tgt_sig, kwargs):
    n_comp = kwargs['n_comp']
    n_pixel = hsi_data.shape[1]
    gmm = GaussianMixture(n_components = n_comp, max_iter = 1, init_params = 'random').fit(hsi_data.T)

    # cluster/assign mixture component to each pixel, get Mahalanobis distance to components
    idx = gmm.predict(hsi_data.T)
    
    return idx, {}
    #gmrx_data = np.zeros(n_pixel)
    #for i in range(n_pixel):
     #   gmrx_data[i] = idx[i]
        
    #return gmrx_data

In [4]:
gmrx_anomaly(hsi_img_sub, n_comp = 8, mask = mask_sub)

/usr/local/anaconda3/envs/bioinfo3/lib/python3.6/site-packages/sklearn/mixture/base.py:237: ConvergenceWarning: Initialization 1 did not converge. Try different init parameters, or increase max_iter, tol or check for degenerate data.
  % (init + 1), ConvergenceWarning)


array([[0.00000000e+000, 1.00000000e+000, 5.00000000e+000, ...,
        2.00000000e+000, 4.00000000e+000, 4.00000000e+000],
       [0.00000000e+000, 5.00000000e+000, 1.00000000e+000, ...,
        1.00000000e+000, 0.00000000e+000, 2.00000000e+000],
       [0.00000000e+000, 0.00000000e+000, 0.00000000e+000, ...,
        2.00000000e+000, 7.00000000e+000, 2.00000000e+000],
       ...,
       [3.00000000e+000, 5.00000000e+000, 0.00000000e+000, ...,
        6.92515209e-310, 6.92515209e-310, 6.92515259e-310],
       [1.00000000e+000, 0.00000000e+000, 1.00000000e+000, ...,
        2.36121535e-001, 3.27630758e-001, 5.15170097e-001],
       [7.00000000e+000, 0.00000000e+000, 1.00000000e+000, ...,
        4.68335762e-310, 4.68335762e-310, 4.68335762e-310]])